In [3]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

import pickle

with open('traintest.pickle', 'rb') as f:
    trainX,testX,trainY,testY = pickle.load(f)

#with open('traintest-smote.pickle','rb') as f:
#    train_res,test_res = pickle.load(f)

with open('ethnic_keys.pickle','rb') as f:
    name_struct_keys,ethnic_keys = pickle.load(f)
        
embedding_vector_length = 1000
lstm_layer = 1000
max_sequence = 50

In [7]:
trainY = np.array([np.where(x>0)[0][0] for x in trainY])

In [4]:
trainX

array([[  207,  3513,  6997, ...,     0,     0,     0],
       [   59,  2867,  9923, ...,     0,     0,     0],
       [  837,  1918,  1396, ...,     0,     0,     0],
       ..., 
       [10016, 11765, 14255, ...,     0,     0,     0],
       [   59,   428,  5269, ...,     0,     0,     0],
       [ 1498,  2819,  2848, ...,     0,     0,     0]], dtype=int32)

In [15]:
#feat_rows = np.zeros([len(trainX),len(name_struct_keys)],dtype=np.bool)
feat_rows = np.zeros([len(trainX),len(name_struct_keys)],dtype=np.int)

In [16]:
for i,x in enumerate(trainX):
    for j in x:
        #feat_rows[i,j-1]=True
        feat_rows[i,j-1]+=1

In [17]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

# use linear SVC for feature selection
lsvc = LinearSVC(C=0.01,penalty='l1',dual=False).fit(feat_rows,trainY)
model =SelectFromModel(lsvc,prefit=True)
X_new = model.transform(feat_rows)

In [18]:
feat_rows.shape

(35653, 62695)

In [19]:
X_new.shape

(35653, 591)

In [54]:
np.where(model.estimator.coef_!=0)[0].shape

(915,)

In [92]:
test = model.estimator.coef_

In [93]:
test.shape

(23, 62695)

In [111]:
selected = np.array([np.sum(test[:,x]!=0) for x in range(test.shape[1])])

In [119]:
selected_coef = np.where(selected>0)[0]

In [126]:
[name_struct_keys[x] for x in selected_coef]

['_E',
 '_Abd',
 'lla_',
 'la__',
 'o_',
 '',
 '_J',
 'ad__',
 '__Bo',
 'h_',
 '_A',
 'man_',
 'an__',
 '__Ab',
 'z_',
 '_S',
 '_Sa',
 'a__A',
 'la_',
 'a_',
 '_I',
 'ou_',
 'u_',
 '_H',
 '_Ha',
 '_El_',
 '_R',
 'ola_',
 'm_',
 'ndra',
 'ra__',
 '__Al',
 '_de_',
 '_Sil',
 'Silv',
 'lva_',
 '_Rod',
 'igue',
 'ues_',
 'es_',
 's_',
 'el__',
 'isti',
 'tina',
 'ina_',
 'y_',
 '_T',
 'ri__',
 '_Jo',
 '_Jos',
 'Jose',
 'ose_',
 'se__',
 'e__M',
 '_Mar',
 'aria',
 'ria_',
 '_Gar',
 'arci',
 'cia_',
 '_M',
 '_Ma',
 'ela_',
 'do_',
 'ro__',
 'na_',
 '_Ali',
 'eh_',
 'med_',
 'ed__',
 'im_',
 'f_',
 '_Mo',
 '_Moh',
 '__Sh',
 '_Sha',
 'd_',
 'gan_',
 'an_',
 'n_',
 'umi_',
 'mi__',
 '__Na',
 'atsu',
 'ne_',
 'e_',
 '_F',
 '_O',
 'ah__',
 '_Al_',
 '_Sh',
 '_Kha',
 'ar_',
 'r_',
 'io__',
 'no_',
 'os__',
 'er_',
 'ia__',
 '__Gr',
 'less',
 'i_',
 '_Ra',
 'nda_',
 'ez__',
 'er__',
 '__Mo',
 '_Mor',
 'eira',
 'ira_',
 'da_',
 'e__L',
 '_Lui',
 'ella',
 'a__S',
 '__Sa',
 '_B',
 'na__',
 'ero_',
 'ro_

In [125]:
selected_coef

array([    0,     8,    13,    14,    22,    23,    24,    28,    30,
          38,    39,    44,    45,    47,    57,    58,    59,    65,
          72,    73,    74,    87,    88,    89,    90,   101,   111,
         115,   122,   135,   137,   138,   145,   154,   155,   157,
         160,   165,   167,   168,   169,   176,   182,   184,   185,
         205,   206,   212,   224,   225,   226,   227,   228,   229,
         231,   233,   234,   244,   246,   248,   250,   251,   257,
         270,   277,   288,   290,   303,   307,   308,   311,   321,
         322,   323,   330,   331,   339,   354,   355,   356,   363,
         364,   366,   371,   383,   384,   385,   397,   405,   407,
         427,   434,   452,   453,   460,   494,   499,   506,   507,
         509,   516,   523,   524,   528,   541,   559,   561,   562,
         566,   567,   574,   575,   577,   590,   591,   592,   601,
         608,   624,   625,   626,   633,   648,   654,   655,   669,
         674,   678,